In [1]:
import numpy as np

data = np.genfromtxt('data.csv',dtype = None, encoding = 'UTF-8', names = True, delimiter = ',')




In [2]:
# preprocessing 
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(data['diagnosis'])
# encoder.transform(data['diagnosis'])
diagnosis = encoder.transform(data['diagnosis']).astype(int)
new_data = diagnosis.reshape(-1,1)
for i in range(2,len(data.dtype.names)):
    new_data = np.concatenate((new_data,np.expand_dims(data[data.dtype.names[i]],axis = 0).T),axis = 1)
new_data[:,1:] = preprocessing.normalize(new_data[:,1:],axis = 0)

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(new_data[:,1:],new_data[:,0],test_size = .2, random_state = 42)

In [22]:
np.save('admin_training',x_train)
np.save('admin_y',y_train)
y = np.expand_dims(y_train, axis = 0)
admin_data = np.concatenate((x_train, y.T), axis = 1)
data_file = open("data_training.csv","w")
for row in admin_data:
    data_file.write(",".join(str(feature) for feature in row)+"\n")
data_file.close()

np.save("admin_data",admin_data)
np.save('client_training',x_test)
np.save('client_y',y_test)

In [16]:
from tempfile import TemporaryFile
tf = TemporaryFile()
np.save(tf,admin_data)
data = np.load(tf)
print(data)

OSError: Failed to interpret file <_io.BufferedRandom name=62> as a pickle

In [105]:
# 10 fold validation
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score,recall_score,f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.externals import joblib

kf = KFold(n_splits = 10)
counter = 1

for train_index, test_index in kf.split(x_train):
    training_samples = x_train[train_index]
    training_y = y_train[train_index]
    test_samples = x_train[test_index]
    test_y = y_train[test_index]
    lda = LinearDiscriminantAnalysis()
    lda.fit(training_samples,training_y)
    predict_y = lda.predict(test_samples)
#     print("test_y: {}".format(test_y))
#     print("predict_y: {}".format(predict_y))
    file = open('model_evaluation_{}.txt'.format(counter),'w+')
    file.write("iteration: {}\n".format(counter))
    file.write("confusion matrix: {}\n".format(confusion_matrix(test_y,predict_y)))
    file.write("precision: {}\n".format(precision_score(test_y,predict_y)))
    file.write("recall: {}\n".format(recall_score(test_y,predict_y)))
    file.write("f1-score: {}\n".format(f1_score(test_y,predict_y)))
    file.close()
    # to store the model
    joblib.dump(lda, 'model_{}'.format(counter))
    counter += 1
    

30